In [32]:
#사용환경을 준비! 참고로 저는 환경변수에 넣었기 때문에(공유가 아니라 제가 이미 돈을 넣어서 제 계정으로 된 API를 쓰고 있습니다!)
#os.getenv를 통해 제 환경변수에 있는 API키 가져오기! 그리고 불필요한 경고문 제거를 위해 warnings 추가
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
import warnings
warnings.filterwarnings(action = 'ignore')

In [33]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader

client = ChatOpenAI(model = 'gpt-4o', temperature = 0.5, max_tokens = 2000)
#PyPDFLoader를 통해 pdf 파일 모델 불러오기!
loader = PyPDFLoader('C:\\Users\\kevinkim\\OneDrive\\바탕 화면\\Sparta_Work\\초거대 언어모델 연구 동향.pdf')
docs = loader.load()

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#요건 RecursiveCharacterTextSplitter: CharacterTextSplitter와 유사하지만, 더 복잡한 분할 방식을 제공
recursive_text_splitter = RecursiveCharacterTextSplitter(
    #chunk_size: 각 청크의 최대 길이
    chunk_size=500,
    #chunk_overlap: 청크 간 겹침의 길이
    chunk_overlap=10,
    #lenth_function: 텍스트 청크의 길이를 계산하는 데 사용하는 함수(기본적으로 len 함수 사용)
    length_function=len,
    #is_separator_regex: 지정한 구분자가 정규식인지 여부를 지정하는 플래그
    is_separator_regex=False,
)
splitted_docs = recursive_text_splitter.split_documents(docs)

#이것은 CharacterTextSplitter: 텍스트를 일정한 길이나 규칙에 따라 분할하는 단순한 방법을 제공. 
#CharacterTextSplitter와 비슷한 방식으로 텍스트를 나누고, 
#나누어진 조각이 여전히 너무 길면 다시 재귀적으로 분할을 시도
#text_splitter = CharacterTextSplitter(
    #separator="\n\n", -> 텍스트 분할 시 사용할 구분자
    #chunk_size=100, -> 설명 완료
    #chunk_overlap=10,-> 설명 완료
    #length_function=len,-> 설명 완료
    #is_separator_regex=False,-> 설명 완료
#)
#splits = text_splitter.split_documents(docs)

In [35]:
#너무 길어서 여기는 주석처리로... enumerate을 통해 index와 각 청크를 따로 분리 후, 
#for i, splitted in enumerate(splitted_docs[:50]):
    #print(f'Chunk {i+1}:')
    #print(splitted)
    #print('=' * 50)

In [36]:
from langchain_openai import OpenAIEmbeddings
from uuid import uuid4
import faiss
from langchain.vectorstores import Chroma
embeddings = OpenAIEmbeddings(model = 'text-embedding-ada-002')

#from_documents를 통해 쉽게 vector_store 생성
vector_store = Chroma.from_documents(documents = splitted_docs, embedding = embeddings)
#심심하니 uuid4를 사용해 이미 chunk로 변한 값들을 고유한 코드로 된 값들로 다시 변경(이러면 PC가 사용하기 더 쉬워진다나 뭐라나...)
uuids = [str(uuid4()) for _ in range(len(splitted_docs))]
vector_store.add_documents(documents = splitted_docs, ids = uuids)

['c5cdbd54-d3ed-4802-8206-369679afda83',
 '744f07b3-125d-4d95-aef5-bd116035f1e5',
 '4d3403e3-ca9f-420a-bc49-36e610070a12',
 'e7a98169-5b54-4d70-8275-d0c542b5bab8',
 'fd11c87d-4971-4e42-9c27-6237e556679a',
 'fbc03635-d981-4def-a61d-da5ebc8dfe9c',
 'ff7abbdb-a76b-4db2-aaaa-d4771d1c8939',
 'c96c6b91-987b-467b-bee5-05c558c4eac6',
 'a8e1ce66-dd0d-47ac-bd64-e8c2e5ec92d9',
 '463b0d9e-7adb-4992-a292-3e77ae9645bd',
 'f0c3a1de-3a42-4427-8b83-7898efd81be4',
 'dc71f5a2-bc83-4897-ba74-ebe0b612fc6f',
 'f0b902ed-2eaf-4441-9075-2374278fb1cc',
 'bebad21d-34c6-427f-a0e7-8292cbfa2432',
 'fc3d8e0f-9c2a-41a7-96ed-1a433ec6db93',
 '6ca6ee32-eaec-421f-842d-57528d6c4348',
 'e428bbd2-3797-42e2-8e27-8e49ca27b6b7',
 '7bbaea5b-3072-49c8-b33b-93431283b955',
 '79091318-d0b7-4908-8c55-3fa61d392eff',
 'a06d0c63-66f8-4276-95a6-39b49a612350',
 '883d8d18-5826-44b1-aba2-a47017f11557',
 'de536e12-6bbe-4103-9355-59530c5cbc62',
 '5cddfafa-2fcb-463b-a331-56ffc118ef6e',
 '81b8a0f5-d516-4144-bd83-1e56aac47282',
 '75925064-3d76-

In [37]:
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
#여기서는 평병하게 retriever를 구성하였습니다.
#Chroma에서 이 부분 llm = client를 통해 쉽게 LLM에 연결이 가능합니다.
retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs = {'k':5})
#여기서는 RetrieveralQA를 써서 질문과 답변을 받을 수 있게 만드는 함수
qa_retriever = RetrievalQA.from_chain_type(
    llm = client,
    chain_type = 'refine',
    retriever = retriever,
    return_source_documents = True,
)

In [38]:
import logging
#만약 질문을 계속 받을시에 작동할 while True: 하지만 비교를 위해 못 쓴다는...
#query로 답을 받아, 그것을 토대로 값을 출
#while True:
query = input('질문을 해주세요:')
#if query == 'exit':
    #print('좋은 대화였습니다!')
    #break
result = qa_retriever(query)
answer_a = result['result']
source_documents = result['source_documents']

    # 결과 출력
print("Answer:", answer_a)
#하지만 QA특성상 추출할 값도 가져와야 하는데, 값을 가져오고 띄우지는 못하게 하는 코드
logging.info("Source Documents: %s", source_documents)

질문을 해주세요: 현재 초거대 언어모델 연구 동향에 대해 설명해줘


C:\Users\kevinkim\AppData\Local\Temp\ipykernel_40864\4200508864.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_retriever(query)


Answer: 현재 초거대 언어모델 연구 동향은 다양한 측면에서 발전하고 있습니다. ChatGPT와 같은 초거대 언어모델의 등장은 자연언어처리(NLP) 분야에서 큰 변화를 가져왔습니다. 첫째로, 이러한 모델들은 번역, 요약, 질의응답, 형태소 분석 등 다양한 자연언어처리 작업을 하나의 모델로 처리할 수 있게 하여 태스크 수렴 현상이 발생하고 있습니다. 이는 기존에 병렬적으로 연구되던 다양한 하위 분야들이 하나의 모델로 통합되고 있다는 것을 의미합니다. 둘째로, 사용자 의도에 맞는 출력을 생성하기 위해 프롬프트를 어떻게 입력하느냐에 따라 모델의 다양한 능력이 창발하는 패러다임이 부각되고 있습니다. 셋째로, 모델의 크기와 복잡성을 계속해서 증가시키며 성능을 향상시키려는 노력이 이어지고 있습니다. 특히, 사전학습 데이터의 다양성을 강조하는 추세가 있으며, 다양한 코퍼스의 최적 혼합 비율과 필요한 데이터 양에 관한 연구가 초기 단계에 있지만 주목받고 있습니다. 예를 들어, GPT-3는 웹페이지뿐만 아니라 책과 뉴스 데이터를 활용했으며, Llama-1 65B 모델에서는 웹페이지 외에도 대화 데이터, 책 및 뉴스, 학술 데이터, 코드 데이터가 포함되어 있습니다. 넷째로, 한국어를 포함한 다양한 언어와 문화적 배경을 포괄할 수 있는 모델 개발이 활발히 진행되고 있으며, 이는 각 언어의 특수성을 반영한 모델 개발을 의미합니다. 다섯째로, 효율적인 학습 및 추론을 위한 새로운 알고리즘과 기술이 개발되고 있습니다. 마지막으로, 초거대 언어모델의 윤리적 사용과 관련된 문제를 해결하기 위한 연구도 중요하게 다루어지고 있습니다. 이러한 동향은 초거대 언어모델의 성능과 활용 가능성을 높이는 데 기여하고 있으며, 연구자 및 관련 전문가들에게 유용한 통찰과 지침을 제공하고자 합니다.


In [39]:
#일반 LLM 모델인 GPT를 이용해 같은 질문을 받아 출력 비교
from openai import OpenAI
client2 = OpenAI()
completion = client2.chat.completions.create(
    model = 'gpt-4o',
    messages = [
        {'role' : 'system', 'content' : '넌 초거대 언어모델 연구 동향에 대해 설명해주는 AI야. 여러 관련 모델들과 지식들을 설명해줘.'},
        {'role' : 'user', 'content' : query}
    ]
)
answer_b = completion.choices[0].message.content
print(answer_b)

초거대 언어 모델에 대한 관심은 급속히 증가하고 있으며, 이러한 모델들은 자연어 처리(NLP) 분야에서 혁신을 불러일으키고 있습니다. 초거대 언어 모델은 수십억 또는 수조 개의 매개변수를 가진 언어 모델로, 대량의 텍스트 데이터를 기반으로 사전 학습하여 다양한 언어 작업을 수행할 수 있습니다. 다음은 이 분야의 주요 연구 동향과 관련 모델들입니다.

1. **GPT (Generative Pre-trained Transformer) 시리즈:** OpenAI에 의해 개발된 이 시리즈는 초거대 언어 모델의 발전을 이끈 대표적인 사례입니다. 다양한 버전이 있으며, 각 버전이 점점 더 큰 모델을 갖추고 더 향상된 언어 이해 및 생성 기능을 제공합니다. GPT-3 및 GPT-4는 각각 매개변수 수를 크게 늘리고, 더 많은 문맥을 이해하며, 다중 작업 수행 능력을 향상시켰습니다.

2. **BERT (Bidirectional Encoder Representations from Transformers):** Google의 BERT는 양방향성을 도입하여 문맥에 대한 이해를 크게 개선한 모델입니다. 이는 비슷한 언어 모델의 개발에 큰 영향을 미쳤으며, 다양한 다운스트림 작업에서 활용되고 있습니다.

3. **T5 (Text-To-Text Transfer Transformer):** Google에서 개발한 T5는 모든 NLP 작업을 텍스트를 입력하고 출력하는 방식으로 통합하여 일관된 접근 방식을 제공합니다. 이는 텍스트 생성, 번역, 요약 등 다양한 작업에 활용됩니다.

4. **MosaicML, EleutherAI와 같은 오픈 소스 노선:** 이들 기관은 접근이 어려운 초거대 모델들을 대중에게 제공하는 데 중점을 두고 있습니다. 특히 EleutherAI의 GPT-Neo 및 GPT-J는 대규모 모델을 공개해 산업 및 학계에서 자유롭게 사용할 수 있도록 했습니다.

5. **다양한 분야에서의 활용 증가:** 이러한 모델들은 고객 서비스, 콘텐츠 생성, 번역, 문서 요약 등 다양한 

In [40]:
#A가 더 좋다는 답변을 받았다! 즉, 내가 만든 값이 더 정확하고 좋다는 뜻!
prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user's instructions and answers the user's question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.

[User Question]
{query}

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]

[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]"""

completion = client2.chat.completions.create(
    model = 'gpt-4o',
    messages = [
        {'role' : 'user', 'content' : prompt}
    ]
)
final = completion.choices[0].message.content
print(final)

Both assistants provided detailed explanations regarding the current research trends in large language models, each touching on various aspects of this rapidly evolving field.  

Assistant A starts by discussing the impact of large language models on the convergence of tasks in natural language processing (NLP) and highlights the trend of emergent abilities depending on the input prompts. Assistant A also delves into efforts to increase model size and complexity, notes the importance of diverse pre-training data, emphasizes the development of models that encompass various languages and cultural backgrounds, and discusses new algorithms for efficient learning and inference. Additionally, Assistant A mentions the ethical considerations surrounding the use of large language models and their implications for researchers and industry experts.

Assistant B begins with an overview of large language models, emphasizing the scale of parameters and their utility in NLP. It then provides specific

In [41]:
"""RAG 모델은 먼저 입력된 질문과 관련된 정보를 검색하고, 그 정보를 바탕으로 답변을 생성합니다. 
이렇게 하면 모델이 정확하고 최신의 정보를 기반으로 답변을 생성할 수 있습니다.
특히 Chroma와 같은 벡터 데이터베이스를 사용하여 RAG 모델을 구현할 때, 모델의 성능을 크게 향상시킬 수 있습니다.
특히 Chroma는 벡터 데이터베이스로, 문서나 텍스트 데이터를 벡터화하여 빠르고 효율적인 검색을 지원하는데, RAG 모델에서 Chroma와 같은 검색 시스템을 사용하면, 
모델이 실시간으로 관련 정보를 빠르게 검색하고, 그 정보를 바탕으로 더 적합한 답변을 생성할 수 있습니다.
한 마디로 LLM 같은 대규모 랭귀지 모델에서 RAG를 통해 최신 정보를 토대로 정확한 답을 생성해낼 수 있기 때문입니다."""

'RAG 모델은 먼저 입력된 질문과 관련된 정보를 검색하고, 그 정보를 바탕으로 답변을 생성합니다. \n이렇게 하면 모델이 정확하고 최신의 정보를 기반으로 답변을 생성할 수 있습니다.\n특히 Chroma와 같은 벡터 데이터베이스를 사용하여 RAG 모델을 구현할 때, 모델의 성능을 크게 향상시킬 수 있습니다.\n특히 Chroma는 벡터 데이터베이스로, 문서나 텍스트 데이터를 벡터화하여 빠르고 효율적인 검색을 지원하는데, RAG 모델에서 Chroma와 같은 검색 시스템을 사용하면, \n모델이 실시간으로 관련 정보를 빠르게 검색하고, 그 정보를 바탕으로 더 적합한 답변을 생성할 수 있습니다.\n한 마디로 LLM 같은 대규모 랭귀지 모델에서 RAG를 통해 최신 정보를 토대로 정확한 답을 생성해낼 수 있기 때문입니다.'

In [42]:
#한번 해보고 싶었던..
from rouge import Rouge
rouge = Rouge()
score = rouge.get_scores(answer_a, answer_b)
print(score)

[{'rouge-1': {'r': 0.125, 'p': 0.18072289156626506, 'f': 0.14778324639763174}, 'rouge-2': {'r': 0.016835016835016835, 'p': 0.02577319587628866, 'f': 0.020366593998034976}, 'rouge-l': {'r': 0.12083333333333333, 'p': 0.1746987951807229, 'f': 0.1428571380232475}}]


In [ ]:
import os
import json
from datetime import datetime
# 디렉토리 경로
PROMPT_DIR = r'C:/Users/kevinkim/Desktop/prompts'
RESULT_DIR = r'C:/Users/kevinkim/Desktop/results'


# 디렉토리 생성
os.makedirs(PROMPT_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

def load_prompt(prompt_name):
    """프롬프트 파일 읽기"""
    prompt_path = os.path.join(PROMPT_DIR, prompt_name)
    print(prompt_path)
    if not os.path.exists(prompt_path):
        raise FileNotFoundError(f"프롬프트 파일 {prompt_name}이(가) 존재하지 않습니다.")
    
    with open(prompt_path, 'r', encoding='utf-8') as file:
        return file.read()

def save_result(prompt_name, result):
    """결과를 파일로 저장"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    result_file_name = f"{os.path.splitext(prompt_name)[0]}_{timestamp}.txt" #prompt1/ .txt -> [0]
    result_path = os.path.join(RESULT_DIR, result_file_name)
    
    with open(result_path, 'w', encoding='utf-8') as file:
        file.write(str(result))
    print(f"결과 저장 완료: {result_path}")

def execute_prompt(prompt_name):
    """프롬프트 실행 및 결과 저장"""
    prompt = load_prompt(prompt_name) 
    try:
        tapping = '=' * 50
        response = qa_retriever(prompt)
        # 결과 문자열 생성
        result = (
            f"{tapping}\n"
            f"Prompt:\n{prompt}\n\n"
            f"Response:\n{response['result']}\n\n"
            f"{tapping}"
        )
        save_result(prompt_name, result)
    except Exception as e:
        print(f"에러 발생: {e}")

files = [f for f in os.listdir(PROMPT_DIR) if f.endswith('.txt')]
for x in files:
    execute_prompt(x)

C:/Users/kevinkim/Desktop/prompts\prompt1.txt
결과 저장 완료: C:/Users/kevinkim/Desktop/results\prompt1_20241119_161902.txt
C:/Users/kevinkim/Desktop/prompts\prompt2.txt
결과 저장 완료: C:/Users/kevinkim/Desktop/results\prompt2_20241119_161956.txt
C:/Users/kevinkim/Desktop/prompts\prompt3.txt


In [96]:
print(f"QA Retriever Response: {response_text}") 

NameError: name 'response_text' is not defined